In [1]:
import fileio
import pydata
import shapes
#import numpy as np
#import pandas as pd

In [6]:
from importlib import reload

In [56]:
reload(fileio)
reload(shapes)

<module 'shapes' from '/home/nelson/Data/kba/shapes.py'>

In [2]:
data = fileio.InputData('arecaceae.csv')
data.iucnFile('arecaceae_categories.csv')

In [3]:
data.groupFiles('arecaceae_groups_assignments.csv', 'arecaceae_group_diversity.csv')

In [16]:
data.taxonGroupsInfo

{'Arecaceae': 3000}

In [17]:
grouplist = []
for igr, gr in ennumerate(sorted(data.taxonGroupsInfo.keys())):
    #
    # Find out minimum spp number to apply criterion B2 for each group
    #
    groupDict = {ix:[data.taxonGroupsInfo[x], 2] for ix,x in enumerate(grouplist)}

In [23]:
spp2groupDict = {}
for ispp, spp in enumerate(sorted(data.taxonGroups.keys())):
    tgr = data.taxonGroups[spp]['group']
    spp2groupDict[ispp] = grouplist.index(tgr)


In [24]:
spp2groupDict

{0: 0,
 1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 0,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0,
 50: 0}

In [68]:
range_threshod = 10000 # If user knows ranges of all species within group, he should select the 25th percentile
for sp in data.points:
    point_list = [x for x in data.points[sp].keys()]
    tarea = shapes.area_estimator(point_list)
    if not data.taxonGroups[sp]['range_size']:
        data.taxonGroups[sp]['range_size'] = tarea

In [ ]:
myar = shapes.KBA('/home/nelson/Dropbox/Humboldt/Postdoc/KBA_by_IUCN/Colombia_KBA', 'SitRecID')

In [ ]:
myar.spp_inclusion(data)

In [ ]:
myar.new_spp_table('test_kba_log.csv')

---

In [ ]:
tilas = data.getTiles(0.2, offsetLat=0.1, offsetLon=0.1)

In [ ]:
for ti in tilas:
    print(data.tile2str(ti))

In [ ]:
mysols = pydata.metasearchAlt(tilas, 0.2, 10000, 10, 50)

In [ ]:
mysols[0][0].aggrScore, mysols[0][0].score, mysols[0][0].ndmScore

In [ ]:
' '.join(map(str, mysols[0][0].spp2crit[36]))

In [ ]:
[(tilas[x].getName(), mysols[0][0].spp2crit[x]) for x in mysols[0][0].spp2crit.keys()]

In [ ]:
rm -r solutions

In [ ]:
shapes.solution2shape(mysols, data)

In [ ]:
procAreas = {}

filehandle = fiona.open('/home/nelson/Data/GIS/Areas_protegidas/RUNAP/runap2Polygon.shp', crs= 'EPSG:4326', encoding = 'utf8')
for item in filehandle:
    #self.polys.append(shape(item['geometry']))
    procAreas[item['properties']['id_pnn']] = {
        'shape': shape(item['geometry'])
        }

In [ ]:
procAreas['20170004']['shape'].contains

In [ ]:
polys = []
for ic in range(mysols[1][0].getSize()):
    if mysols[1][0].getValue(ic) > 0:
        y, x = irkeys[ic]
        xBase = myfile.originN[0] + myfile.cellSize * x
        yBase = myfile.originN[1] - myfile.cellSize * y
        ocor = [(xBase + myfile.cellSize, yBase),
            (xBase, yBase),
            (xBase, yBase - myfile.cellSize),
            (xBase + myfile.cellSize, yBase - myfile.cellSize)]
        polys.append(Polygon(ocor))
solpoly = unary_union(polys)

for sp in myfile.points:
    for lon, lat in myfile.points[sp]:
        if solpoly.contains(Point(lon, lat)):
            print (sp)


In [ ]:
solpoly.contains

In [ ]:
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int',
                  'score': 'float',
                  'NDMscore': 'float'},
    }

irkeys = list(myfile.index_reg.keys())

for igr, gr in enumerate(mysols):
    solpolcoll = []
    multipol = None
    filename = 'group_{0}.shp'.format(igr)

    for its,  tsol in enumerate(gr):
        polys = []
        solpoly = None
        for ic in range(tsol.getSize()):
            if tsol.getValue(ic) > 0:
                y, x = irkeys[ic]
                xBase = myfile.originN[0] + myfile.cellSize * x
                yBase = myfile.originN[1] - myfile.cellSize * y
                ocor = [(xBase + myfile.cellSize, yBase),
                    (xBase, yBase),
                    (xBase, yBase - myfile.cellSize),
                    (xBase + myfile.cellSize, yBase - myfile.cellSize)]
                polys.append(Polygon(ocor))
        solpoly = unary_union(polys)
        
        if its == 0:
                with fiona.open(filename, 'w', 'ESRI Shapefile', schema, from_epsg(4326)) as c:
                    c.write({'geometry': mapping(solpoly),
                        'properties': {'id': ip,
                                      'score':  tsol.aggrScore,
                                      'NDMscore': tsol.ndmScore}
                            })

        else:
            with fiona.open(filename, 'a', 'ESRI Shapefile', schema, from_epsg(4326)) as c:
                c.write({'geometry': mapping(solpoly),
                    'properties': {'id': ip,
                                      'score':  tsol.aggrScore,
                                      'NDMscore': tsol.ndmScore}
                        })


In [ ]:
for spp in [tilas[x].getName() for x in mysols[0][0].spp2crit.keys()]:
    print(spp, len(data.points[spp]))

In [ ]:
print(data.tile2str( mysols[1][0]))

In [ ]:
mysols = pydata.metasearchAlt(tilas, 0.2, 1000, 500, 2)

for ig, group in enumerate(mysols):
    print ('#' * 50)
    print ('Group {0}'.format(ig))
    for iso, so in enumerate(group):
        print("Solution {0} of {1}".format(iso, len(group)))
        print(so.score, "-", so.ndmScore, "-", so.aggrScore)
        #print("Islands: ", pydata.islNum(so))
        #if not pydata.isCont(so):
        #    print("NOT CONTINUOUS!!!")
        #print(so.toBitList())
        print(myfile.tile2str(so))
    